In [1]:
import numpy as np
import numpy.linalg as alg
import math
import pandas as pd
from MethodsFromArticle import predict
from MethodsFromArticle import predict_grad

# Metrics

In [9]:
def MAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(true[i][j])
            elif result[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(result[i][j])
    return 100*answer / (true.shape[0] * true.shape[1])

In [10]:
def WAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += (abs(true[i][j]) / np.sum(true)) * abs(result[i][j] - true[i][j]) / abs(true[i][j])
            else:
                answer += abs(result[i][j] - true[i][j]) / np.sum(true)
    return answer * 100

In [11]:
def SWAD(result, true):
    return np.sum(np.abs(true*(result - true)))  / np.sum(np.square(true))

## Full circle of predictions for all methods from "Projection of Supply and Use Tables: Methods and their Empirical Assessment"

In [5]:
metrics_name = ['MAPE', 'WAPE', 'SWAD']
methods_name = ['IWS', 'INS', 'ISD', 'RAS']
metrics = [MAPE, WAPE, SWAD]

In [6]:
sup10 = np.load('data//sup10.npy')
sup11 = np.load('data//sup11.npy')
sup12 = np.load('data//sup12.npy')
use10 = np.load('data//use10.npy')
use11 = np.load('data//use11.npy')
use12 = np.load('data//use12.npy')

In [9]:
total_results = {j:{i: {method: {metric: 0 for metric in metrics_name} 
                     for method in methods_name} 
                 for i in ['sup11', 'sup12', 'use11', 'use12']} for j in ['0', '10', '100', '1000', '10000', '100000']}

In [16]:
for M in ['0', '10', '100', '1000', '10000', '100000']:
    for method in methods_name:
        if method == 'IWS':
            continue
        eps = 1e-8
        pred_sup11 = predict(sup10, sup11, method=method, M=int(M), eps=eps)
        pred_sup12 = predict(pred_sup11, sup12, method=method, M=int(M), eps=eps)
        pred_use11 = predict(use10, use11, method=method, M=int(M), eps=eps)
        pred_use12 = predict(pred_use11, use12, method=method, M=int(M), eps=eps)
        for metric_name, metric in zip(metrics_name, metrics):
            total_results[M]['sup11'][method][metric_name] = metric(pred_sup11, sup11)
            total_results[M]['sup12'][method][metric_name] = metric(pred_sup12, sup12)
            total_results[M]['use11'][method][metric_name] = metric(pred_use11, use11)
            total_results[M]['use12'][method][metric_name] = metric(pred_use12, use12)
    method = 'IWS'
    eps = 0.01
    pred_sup11 = predict(sup10, sup11, method=method, M=int(M), eps=eps)
    pred_sup12 = predict(np.where(pred_sup11 > 0, pred_sup11, 0), sup12, method=method, M=int(M), eps=eps)
    pred_use11 = predict(use10, use11, method=method, M=int(M), eps=eps)
    pred_use12 = predict(np.where(pred_use11 > 0, pred_use11, 0), use12, method=method, M=int(M), eps=eps)
    for metric_name, metric in zip(metrics_name, metrics):
        total_results[M]['sup11'][method][metric_name] = metric(pred_sup11, sup11)
        total_results[M]['sup12'][method][metric_name] = metric(pred_sup12, sup12)
        total_results[M]['use11'][method][metric_name] = metric(pred_use11, use11)
        total_results[M]['use12'][method][metric_name] = metric(pred_use12, use12)

In [17]:
for M in ['0', '10', '100', '1000', '10000', '100000']:
    print('Results with M = ' + M)
    for instance in ['sup11', 'sup12', 'use11', 'use12']:
        print('-----------------------'+instance+'--------------------')
        print(pd.DataFrame(total_results[M][instance]))
        print('------------------------------------------------')
        print()

Results with M = 0
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  852.871567  7.583174  191.204137  81.943565
WAPE   16.025143  1.700187   10.940347   5.827249
SWAD    0.052394  0.004038    0.046383   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  729.721125  9.755207  306.896772  83.090178
WAPE   22.099099  2.381380   14.609489   5.119052
SWAD    0.070144  0.004930    0.060405   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  941.412129  11.993685  217.577982  33.427618
WAPE   25.348919   5.013336   17.466787   9.515708
SWAD    0.133537   0.021530    0.125710   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        INS    

## Full circle of predictions using proximal gradient method

In [2]:
results_grad = {key: {metric: 0 for metric in ['MAPE', 'WAPE', 'SWAD']} for key in ['sup11', 'sup12', 'use11', 'use12']}

In [7]:
pred_sup11 = predict_grad(sup10, sup11)
pred_sup12 = predict_grad(pred_sup11, sup12)
pred_use11 = predict_grad(use10, use11)
pred_use12 = predict_grad(pred_use11, use12)

In [12]:
for metric_name, metric in zip(['MAPE', 'WAPE', 'SWAD'], [MAPE, WAPE, SWAD]):
    results_grad['sup11'][metric_name] = metric(pred_sup11, sup11)
    results_grad['sup12'][metric_name] = metric(pred_sup12, sup12)
    results_grad['use11'][metric_name] = metric(pred_use11, use11)
    results_grad['use12'][metric_name] = metric(pred_use12, use12)

In [13]:
pd.DataFrame(results_grad)

,sup11,sup12,use11,use12
MAPE,7.139003,9.194080,10.963089,13.160047
WAPE,1.666577,2.376965,6.067734,8.286170
SWAD,0.002962,0.004270,0.032906,0.051979
